In [1]:
pip install flask

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.2/101.2 KB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 KB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 KB 6.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import time

In [213]:
%%writefile server.py

from collections import Counter
from flask import Flask
from flask import jsonify
from flask import request
from flask import abort

def read_dictionary(filename: str) -> Counter:
    '''
        Read dictionary file with words statistics.
        Function results is Counter datatype.
    '''
    #YOUR CODE HERE
    words = dict()
    with open(filename, 'r') as f:
        for line in f:
            key, val = line.split()
            words[key] = int(val)

    return Counter(words)

WORDS = read_dictionary('dictionary.txt')


def P(word, N=sum(WORDS.values())): 
    '''
        Probability of `word`: (num occurances of `word`)/ (total count of words) 
    '''
    # YOUR CODE HERE
    return WORDS[word]/N

def most_probable(word): 
    '''
        Find most probable (with max ) spelling correction for word. 
        Hint: see max function + key param 
            https://www.programiz.com/python-programming/methods/built-in/max
    '''
    # YOUR CODE HERE
    return max(candidates(word), key=lambda w: P(w))

def candidates(word): 
    '''
        Generate most nearest spelling corrections for word.
        If found word in dictionary then return word, otherwise
        try found words from one and then two edit distance
    '''
    one_symbol_change = generate_candidates_one_symbol(word)
    two_symbol_change = generate_candidates_two_symbol(word)
    return (known([word]) or known(one_symbol_change) or known(two_symbol_change) or [word])

def known(words): 
    '''
        The subset of `words` that appear in the dictionary of WORDS.
    '''
    return set(w for w in words if w in WORDS)

def generate_candidates_one_symbol(word):
    '''
        Generate candidates that are one edit symbol away from `word`.
    '''
    
    letters    = 'абвгдеёжзиклмнопрстуфхцчшщъыьэюя'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def generate_candidates_two_symbol(word): 
    '''
        Generate all сandidates that are two edits away from `word`.
    '''
    return [
        e2 for e1 in generate_candidates_one_symbol(word)
        for e2 in generate_candidates_one_symbol(e1)
    ]

app = Flask(__name__)

@app.route('/version', methods=['GET'])
def get_version():
    # YOUR CODE HERE
    return jsonify({'version':'1.0'})

@app.route('/correct', methods=['GET'])
def correct():
    # YOUR CODE HERE
    check_word = request.args.get('check_word')
    corrected = most_probable(check_word)
    return jsonify({'correct_word': corrected})

@app.route('/add_word', methods=['POST'])
def add_word():
    # YOUR CODE HERE
    added_word =  request.json['added_word']
    WORDS[added_word] += 1

    return jsonify(success=True)

@app.route('/candidates/<int:edit_distance>', methods=['GET'])
def get_candidates(edit_distance: int):
    # YOUR CODE HERE
    word =  request.args.get('word')
    one_symbol_change = generate_candidates_one_symbol(word)
    two_symbol_change = generate_candidates_two_symbol(word)
    responses = {
        1: {'words': list(set.union(known([word]), known(one_symbol_change), [word]))},
        2: {'words': list(set.union(known([word]), known(one_symbol_change), known(two_symbol_change), [word]))}
    }
    
    return jsonify(responses[edit_distance]) if edit_distance in responses else abort(404)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5555)

Overwriting server.py


In [215]:
resp = requests.get("http://localhost:5555/version")

assert resp.status_code == 200,  f'''Статус некорректный статус ответа: {resp.status_code}.
                                     Описание ошибки: {resp.reason}.
                                     Посмотрите в запущенном терминале более детальную информацию о ней'''
assert resp.json().get("version") == "1.0", "Некорректный номер версии"
resp.json()

{'version': '1.0'}

In [216]:
params = {"check_word": "полкводиц"}
resp = requests.get("http://localhost:5555/correct", params=params)

assert resp.status_code == 200, "Код ошибки не равен 200, ошибки на стороне сервера"
assert resp.json()['correct_word'] == "полководец", "Исправление некорретное"
resp.json()['correct_word']

'полководец'

In [217]:
params = {"check_word": "радио"}
resp = requests.get("http://localhost:5555/correct", params=params)

print(resp.json())

{'correct_word': 'ради'}


In [218]:
# Проверим работоспособность метода добавления нового слова
data = {"added_word": "радио"}
resp = requests.post("http://localhost:5555/add_word", json=data)

assert resp.status_code == 200, "Код ошибки не равен 200, ошибки на стороне сервера"

# Проверим, что новое слово действительно добавилось в словарь

params = {"check_word": "радио"}
resp = requests.get("http://localhost:5555/correct", params=params)


assert resp.status_code == 200, "Код ошибки не равен 200, ошибки на стороне сервера"
assert resp.json()['correct_word'] == "радио", "Новое слово радио не было добавлено в словарь корректных слов"


print(resp.json())

{'correct_word': 'радио'}


In [219]:
resp = requests.get("http://localhost:5555/candidates/1", params={"word": "генерал"})
assert resp.status_code == 200, "Код ответа не равен 200, в решении присутствуют ошибки"
print("Ответ от сервера:", resp.json())
assert len(resp.json()["words"]) == 5, f'''Ошибка в логике работы, для данного
                слова должно быть сформировано 5 кандидатов, получено {len(resp.json()['words'])}'''

resp = requests.get("http://localhost:5555/candidates/2", params={"word": "генерал"})
assert resp.status_code == 200, "Код ответа не равен 200, в решении присутствуют ошибки"
print("Ответ от сервера:", resp.json())
assert len(resp.json()["words"]) == 9, f'''Ошибка в логике работы, для данного слова должно 
                быть сформировано 9 кандидатов, получено {len(resp.json()['words'])}'''

Ответ от сервера: {'words': ['генерал', 'генералы', 'генерала', 'генералу', 'генерале']}
Ответ от сервера: {'words': ['генералом', 'генералам', 'генерала', 'генералу', 'генерале', 'генерал', 'генералах', 'генералы', 'генералов']}


In [220]:
%%writefile Dockerfile

# Используем в качестве базового образа образ python
FROM python:3.11-slim

# Создайте папку, в которой будет храниться исходные файлы приложения, с именем app
# YOUR CODE HERE
RUN mkdir /app 

# Пометьте созданную папку app как рабочую директорию. 
# YOUR CODE HERE
WORKDIR /app

# Перекопируйте файл server.py с текущей директории в созданную выше папку app
# YOUR CODE HERE
COPY server.py /app
# Перекопируйте файл словаря c текущей директории в созданную выше папку app
# YOUR CODE HERE
COPY dictionary.txt /app

# Установим библиотеку Flask внутрь контейнера
RUN pip3 install -q Flask

# Сделаем порт 5555, на котором работет приложение видимым
EXPOSE 5555

# Установим команду, которая будет запускаться при старте котнейнера
CMD ["python3", "server.py"]

Writing Dockerfile


In [221]:
# Удаление старого образа
! docker rmi corrector:1.0
# Создание нового
! docker build --progress=plain --no-cache --rm=true  -t corrector:1.0 .

Error response from daemon: No such image: corrector:1.0
#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile:
#1 transferring dockerfile: 1.20kB done
#1 DONE 0.2s

#2 [internal] load .dockerignore
#2 transferring context: 2B done
#2 DONE 0.2s

#3 [internal] load metadata for docker.io/library/python:3.11-slim
#3 DONE 2.1s

#4 [internal] load build context
#4 transferring context: 973.76kB 0.0s done
#4 DONE 0.0s

#5 [1/6] FROM docker.io/library/python:3.11-slim@sha256:d11b9bd5e49ea7401753d78f4d3b56f3aec952b85b49bcae88981f0452818e0b
#5 resolve docker.io/library/python:3.11-slim@sha256:d11b9bd5e49ea7401753d78f4d3b56f3aec952b85b49bcae88981f0452818e0b 0.0s done
#5 sha256:d11b9bd5e49ea7401753d78f4d3b56f3aec952b85b49bcae88981f0452818e0b 1.65kB / 1.65kB done
#5 sha256:9b394f5b91f048ac4121d64c69028178a7728ffed9f59fba27f62d4f6cc29f97 1.37kB / 1.37kB done
#5 sha256:da34282a1612d67286f90455408c3364e3258c1cd9a873107

In [224]:
!docker image ls

REPOSITORY    TAG       IMAGE ID       CREATED          SIZE
corrector     1.0       0039c29e703b   35 seconds ago   148MB
ubuntu        latest    174c8c134b2a   5 weeks ago      77.9MB
ubuntu        <none>    e4c58958181a   3 months ago     77.8MB
ubuntu        <none>    3565a89d9e81   3 months ago     77.8MB
hello-world   <none>    9c7a54a9a43c   8 months ago     13.3kB
hello-world   latest    d2c94e258dcb   8 months ago     13.3kB
hello-world   linux     d2c94e258dcb   8 months ago     13.3kB


In [225]:
# Останавливаем старый конейнер с приложением 
! if [ "$(docker ps | grep -c 'corrector_app')" -gt 0 ]; then docker stop corrector_app; fi

# Запускаем новый контейнер
! docker run --name corrector_app --rm -d -p 8000:5555  corrector:1.0;

# Прежде чем слать запросы на веб-сервер подождём 2 секунды, пока он запустится
time.sleep(2.0)

try:
    resp = requests.get("http://localhost:8000/candidates/1", params={"word": "генерал"})
except Exception as ex:
    print("\033[91m В решении ошибка присутствует ошибка.", ex, "\033[0m")

assert resp.status_code == 200, "Код ответа не равен 200, в решении присутствуют ошибки"

if resp.status_code == 200:
    print("\033[92mВаш контейнеризированное приложение корректно работает!\033[0m")

5b9539037ddbbadc130d83329ca7c92e3f3f771dd14ef1936247b7180bf7751a
Ваш контейнеризированное приложение корректно работает!


In [226]:
!docker container ls

CONTAINER ID   IMAGE           COMMAND               CREATED          STATUS          PORTS                                       NAMES
5b9539037ddb   corrector:1.0   "python3 server.py"   51 seconds ago   Up 50 seconds   0.0.0.0:8000->5555/tcp, :::8000->5555/tcp   corrector_app
